In [3]:
import numpy as np
import time
import datetime
import json
import pandas as pd

import mysql.connector
from mysql.connector import Error

from bokeh.plotting import *
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, LogColorMapper, ColorMapper, LogTicker, ColorBar, BasicTicker, ColorBar, LinearColorMapper, PrintfTickFormatter
from bokeh.models.widgets import Tabs, Panel
# from .chart_constants import (PLOT_FORMATS, ORANGE, BLUE, DARK_GRAY, AXIS_FORMATS, ORANGE_SHADOW, 
#                              FONT_PROPS_SM, FONT_PROPS_MD, FONT_PROPS_LG, GREEN)
from bokeh.io import show, output_notebook
import bokeh.palettes as bp
from bokeh.models.glyphs import Text
# import hail as hl

import urllib
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)

In [4]:
json_url = urllib.request.urlopen('https://api.helioviewer.org/?action=getDataSources')
hv_keys = json.loads(json_url.read())

In [6]:
start_time=time.time()
try:
    connection = mysql.connector.connect(host='localhost',
                                         database='hv',
                                         user='hv_varun',
                                         password='Helioviewer@2020')

    sql_select_Query = "SELECT filepath, date, sourceid FROM data WHERE sourceId=%s;"%hv_keys['SDO']['AIA']['1600']['sourceId']
#     sql_select_Query = "SELECT count(*) FROM data WHERE filepath LIKE '/AIA/1600/%';"
#     sql_select_Query = "SELECT * FROM data LIMIT 20;"
    cursor = connection.cursor()
    cursor.execute(sql_select_Query)
    records = cursor.fetchall()
    print("Total number of rows in data is: ", cursor.rowcount)

    print("\nPrinting each laptop record")
#     for row in records:
#         print("Index = ", row[0])
#         print("Location = ", row[1])
#         print("Filename  = ", row[2])
#         print("OBS_DATE  = ", row[3], "\n")

except Error as e:
    print("Error reading data from MySQL table", e)
finally:
    if (connection.is_connected()):
        connection.close()
        cursor.close()
        print("MySQL connection is closed", time.time()-start_time )

Total number of rows in data is:  7056352

Printing each laptop record
MySQL connection is closed 108.46920776367188


In [7]:
hv = pd.DataFrame(records, columns=cursor.column_names)
hv = hv.sort_values('date').reset_index(drop=True)
hv

,filepath,date,sourceid
0,/AIA/1600/2010/06/02,2010-06-02 00:05:30,15
1,/AIA/1600/2010/06/02,2010-06-02 00:05:54,15
2,/AIA/1600/2010/06/02,2010-06-02 00:06:18,15
3,/AIA/1600/2010/06/23,2010-06-23 00:00:17,15
4,/AIA/1600/2010/06/23,2010-06-23 00:00:41,15
...,...,...,...
7056347,/AIA/2020/08/03/1600,2020-08-03 13:59:26,15
7056348,/AIA/2020/08/03/1600,2020-08-03 14:00:14,15
7056349,/AIA/2020/08/03/1600,2020-08-03 14:01:02,15
7056350,/AIA/2020/08/03/1600,2020-08-03 14:01:50,15


In [98]:
day_bins = pd.date_range(hv['date'].min()-pd.DateOffset(days=1), hv['date'].max()+pd.DateOffset(days=1), freq='D').to_period('D').to_timestamp()

In [99]:
day_bins

DatetimeIndex(['2010-06-01', '2010-06-02', '2010-06-03', '2010-06-04',
               '2010-06-05', '2010-06-06', '2010-06-07', '2010-06-08',
               '2010-06-09', '2010-06-10',
               ...
               '2020-07-26', '2020-07-27', '2020-07-28', '2020-07-29',
               '2020-07-30', '2020-07-31', '2020-08-01', '2020-08-02',
               '2020-08-03', '2020-08-04'],
              dtype='datetime64[ns]', length=3718, freq='D')

In [106]:
counts, bins = np.histogram(hv['date'], bins = day_bins)#.size+1, range=(day_bins[0], day_bins[-1]))
hist, edges = np.histogram(counts, bins=range(0,counts.max(),100))

In [183]:
from bokeh.models import Span

In [188]:
def interactive_histogram(df, n_bins, title, x_axis_label, x_tooltip, log=None):
    
    import pandas as pd
    import numpy as np
    from bokeh.plotting import figure

    from bokeh.io import show, output_notebook
    from bokeh.models import ColumnDataSource, HoverTool, CategoricalColorMapper
    from bokeh.palettes import Category10_5, Category20_16

    arr_hist, edges = np.histogram(df, bins=n_bins)

    # Column data source
    arr_df = pd.DataFrame({'count': arr_hist, 'left': edges[:-1], 'right': edges[1:]})
    total = arr_df['count'].sum()
    arr_df['f_count'] = ['%d' % count for count in arr_df['count']]
    arr_df['f_percent'] = ['%.2f%%' %(count/total*100) for count in arr_df['count']]
    arr_df['f_interval'] = ['%d to %d ' % (left, right) for left, right in zip(arr_df['left'], arr_df['right'])]
    # column data source
    arr_src = ColumnDataSource(arr_df)
    
    cumulative_data = np.cumsum(arr_hist)
#     cumulative_data = np.append(cumulative_data, cumulative_data[-1])
    df_cum = pd.DataFrame({'count_cum': cumulative_data, 'x': np.arange(0,df.max(),100)[1:]})
    cum_src = ColumnDataSource(df_cum)
        
    # Set up the figure same as before
    panels = []
    for axis_type in ["linear", "log"]:
        p = figure(y_axis_type=axis_type,
                   title = title,
                   x_axis_label = x_axis_label, 
                   y_axis_label = 'Day count',
                   background_fill_color="#fafafa")
        # Add a quad glyph with source this time
        r1 = p.quad(bottom=0.95, 
                   top='count', 
                   left='left', 
                   right='right', 
                   source=arr_src,
                   fill_color='navy',
                   alpha=0.5,
    #                hover_fill_color='navy',
                   hover_fill_alpha=0.2,
                   line_color='white',
                   legend='Histogram')
        
        # Add style to the plot
        p.title.align = 'center'
        p.title.text_font_size = '18pt'
        p.xaxis.axis_label_text_font_size = '12pt'
        p.xaxis.major_label_text_font_size = '12pt'
        p.yaxis.axis_label_text_font_size = '12pt'
        p.yaxis.major_label_text_font_size = '12pt'
        p.grid.grid_line_color="white"
        
        text_source = ColumnDataSource(dict(x=[2000],y=[1500],text=['Total Day Count = %d'%total]))
        glyph = Text(x="x", y="y", text="text", text_color="black")
        p.add_glyph(text_source, glyph)
        # Add a hover tool referring to the formatted columns
        hover = HoverTool(renderers = [r1], tooltips = [(x_tooltip, '@f_interval'),
                                                        ('Day count', '@f_count'),
                                                        ('Day count percentage', '@f_percent')])

        # Add the hover tool to the graph
        p.add_tools(hover)
        
        r2 = p.line(x='x', y='count_cum', line_color='#036564', line_width=1, source=cum_src, hover_line_alpha=0.5, legend="Cumulative distribution")
        p.circle(x='x', y='count_cum', line_color='#036564', line_width=3, source=cum_src, legend="Cumulative distribution" )
        hover = HoverTool(renderers=[r2], line_policy='nearest', tooltips = [(x_tooltip, '@x'), 
                                                      ('Cumulative Day count', '@count_cum')])
        
        p.ray(x=[1800], y=[200], length=0, angle=np.pi)
        p.add_layout(Span(location=1800, dimension='height', legend='Expected date file count'))
        # Add the hover tool to the graph
        p.add_tools(hover)
        p.legend.location = "top_left"
        panel = Panel(child=p, title=axis_type)
        panels.append(panel)
    tabs = Tabs(tabs=panels)

    return tabs

In [189]:
output_file('AIA1600_histogram.html')

df = counts
n_bins = range(0,counts.max(),100)
title = 'Histogram for AIA 1600'
x_axis_label = 'No. of Data files'
x_tooltip = '#Data files'
btabs = interactive_histogram(df,n_bins,title,x_axis_label,x_tooltip,log='log')
show(btabs)

AttributeError: unexpected attribute 'legend' to Span, possible attributes are dimension, js_event_callbacks, js_property_callbacks, level, line_alpha, line_cap, line_color, line_dash, line_dash_offset, line_join, line_width, location, location_units, name, render_mode, subscribed_events, tags, visible, x_range_name or y_range_name

In [115]:
(counts<1800).sum()

1260

In [118]:
arr_hist, edges = np.histogram(counts, bins=n_bins)

In [120]:
arr_hist, edges

(array([  84,    1,    1,    4,    3,    7,    3,    6,    8,   11,    5,
           4,    9,   15,   13,   19,   73,  994, 1598,   16,   18,   34,
          71,  364,  174,  109,   30,    6,    3,    6,    2,    6,    7,
           2,    2]),
 array([   0,  100,  200,  300,  400,  500,  600,  700,  800,  900, 1000,
        1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100,
        2200, 2300, 2400, 2500, 2600, 2700, 2800, 2900, 3000, 3100, 3200,
        3300, 3400, 3500]))

In [124]:
(counts<100).sum()

84